In [1]:
import copy

import pandas as pd
from nltk.corpus import floresta as flt
from nltk.corpus import machado as mch
from nltk.corpus import mac_morpho as mcm

from utils import load_six_emotions, load_valence_emotions_from_oplexicon, load_valence_emotions_from_sentilex, generate_corpus
from unsupervised.emotional_lsa import EmotionalLSA

In [5]:
original_phrases = [
    'Quando a tristeza bater na sua porta, abra um belo sorriso e diga: Desculpa, mas hoje a felicidade chegou primeiro!',
    'Feliz é aquele que vê a felicidade dos outros sem ter inveja. O sol é para todos e a sombra pra quem merece.',
    'Minha meta é ser feliz, não perfeito.',
    'Ser feliz até onde der, até onde puder. Sem adiar, ser feliz o tanto que durar.',
    'Nunca deixe ninguém dizer que você não pode fazer alguma coisa. Se você tem um sonho, tem que correr atrás dele. As pessoas não conseguem vencer, e dizem que você também não vai vencer. Se quer alguma coisa, corre atrás.',
    'O maior problema em acreditar nas pessoas erradas, é que um dia você acaba não acreditando em mais ninguém.',
    'Não me deixe ir, posso nunca mais voltar.',
    'Não me arrependo de ter conhecido ninguém, só me arrependo de ter perdido tanto tempo com algumas pessoas.',
    'Se existe uma coisa que eu me arrependo é de ter confiado em algumas pessoas.',
    'Prefiro que enxerguem em mim erros com arrependimento do que uma falsa perfeição.',
    'Mesmo sabendo que um dia a vida acaba, a gente nunca está preparado para perder alguém.',
    'A morte é uma pétala que se solta da flor e deixa uma eterna saudade no coração.',
    'Mãe é imortal, porque quando ela parte para outro mundo fica vivendo nas lágrimas que escorrem em nosso rosto eternamente.',
    'Só existem dois motivos pra uma pessoa se preocupar com você: Ou ela te ama muito, ou você tem algo que ela queira muito!',
    'Ser feliz nao é ter uma vida perfeita, mas sim reconhecer que vale a pena viver apesar de todos os desafios e perdas.',
    'Minha maravilhosa vida é uma merda.'
]
phrases = copy.deepcopy(original_phrases)
phrases += [' '.join(f).replace('_', ' ') for f in flt.sents()[:1000]]
# phrases += [' '.join(f).replace('_', ' ') for f in mch.sents()[:500]]
phrases += [' '.join(f).replace('_', ' ') for f in mcm.sents()[:1000]]

filename = '/home/rdenadai/programas/vagrant_img/vagrant.machine.puphpet/projetos/github/sentiment-analysis-2018-president-election/dataset/tweets-tratados-polaridade-10-10-2018.csv'
with open(filename, 'r') as h:
    tweets = h.readlines()
    for tweet in tweets[1:5]:
        print(tweet.split(';'))

phrases = generate_corpus(phrases)
ldocs = [f'D{i}' for i in range(len(original_phrases))]

['387164512995930113', 'WildCat, o robot de quatro patas que corre a 25km/hora - ', 'Neutral\n']
['387164677152964608', 'Parem de olhar para o meu telemovel quando estou a tweetar :)', 'Positive\n']
['387165113347608576', '@scrismelo Porque ainda não acordaste?!', 'Neutral\n']
['387165245263073280', '@TejoPaulo boa opção!', 'Neutral\n']
Iniciando processamento...
Filtro e finalização...
Finalizado...


In [9]:
print('Loading emotional words: ')
emotion_words = load_six_emotions('/home/rdenadai/programas/vagrant_img/vagrant.machine.puphpet/projetos/github/sentiment-analysis-2018-president-election/dataset/emocoes')
emotion_words_n = load_valence_emotions_from_oplexicon('/home/rdenadai/programas/vagrant_img/vagrant.machine.puphpet/projetos/github/sentiment-analysis-2018-president-election/dataset/emocoes/oplexicon_v3.0/lexico_v3.0.txt')
emotion_words['NEUTRO'] = emotion_words_n['NEUTRO']

print('Starting EmotionalLSA model...')
model = EmotionalLSA(debug=True)
print('fit and transform...')
predicted = model.fit_transform(phrases, emotion_words)

print('\nShowing results:')
print('-' * 40)
for i, frase in enumerate(original_phrases):
    print(f' D{i} - {frase}')

df = pd.DataFrame(predicted[:len(original_phrases)], index=ldocs, columns=emotion_words.keys())
display(df.head(15))

Loading emotional words: 
Starting EmotionalLSA model...
fit and transform...
Count:
Actual number of features: 6313
Processing emotional state... this may take a while...

Showing results:
----------------------------------------
 D0 - Quando a tristeza bater na sua porta, abra um belo sorriso e diga: Desculpa, mas hoje a felicidade chegou primeiro!
 D1 - Feliz é aquele que vê a felicidade dos outros sem ter inveja. O sol é para todos e a sombra pra quem merece.
 D2 - Minha meta é ser feliz, não perfeito.
 D3 - Ser feliz até onde der, até onde puder. Sem adiar, ser feliz o tanto que durar.
 D4 - Nunca deixe ninguém dizer que você não pode fazer alguma coisa. Se você tem um sonho, tem que correr atrás dele. As pessoas não conseguem vencer, e dizem que você também não vai vencer. Se quer alguma coisa, corre atrás.
 D5 - O maior problema em acreditar nas pessoas erradas, é que um dia você acaba não acreditando em mais ninguém.
 D6 - Não me deixe ir, posso nunca mais voltar.
 D7 - Não me 

,ALEGRIA,DESGOSTO,MEDO,RAIVA,SURPRESA,TRISTEZA,NEUTRO
D0,15.29,27.94,12.60,5.03,-2.06,-6.94,2.43
D1,8.11,-0.32,9.04,-7.05,-0.64,-2.59,35.52
D2,-5.88,-0.30,1.97,5.07,7.46,10.22,-3.18
D3,13.67,17.54,3.13,-15.93,2.67,18.48,-9.12
D4,26.46,8.41,-20.45,-5.15,6.61,22.49,53.21
D5,8.32,31.75,3.30,-8.95,-0.16,3.57,-6.35
D6,2.07,7.81,3.79,-8.49,10.78,-8.25,-5.51
D7,9.07,11.64,-12.39,15.85,19.49,36.55,50.53
D8,-16.62,25.58,12.72,27.67,7.83,16.76,17.91
D9,-2.85,11.02,2.21,1.42,-7.44,0.86,0.48


In [7]:
print('Loading emotional words: ')
emotion_words = load_valence_emotions_from_oplexicon('/home/rdenadai/programas/vagrant_img/vagrant.machine.puphpet/projetos/github/sentiment-analysis-2018-president-election/dataset/emocoes/oplexicon_v3.0/lexico_v3.0.txt')

print('Starting EmotionalLSA model...')
model = EmotionalLSA(debug=True)
print('fit and transform...')
predicted = model.fit_transform(phrases, emotion_words)

print('\nShowing results:')
print('-' * 40)
for i, frase in enumerate(original_phrases):
    print(f' D{i} - {frase}')

size = len(original_phrases)
df = pd.DataFrame(predicted[:size], index=ldocs, columns=emotion_words.keys())
display(df)

Loading emotional words: 
Starting EmotionalLSA model...
fit and transform...
Count:
Actual number of features: 6313
Processing emotional state... this may take a while...

Showing results:
----------------------------------------
 D0 - Quando a tristeza bater na sua porta, abra um belo sorriso e diga: Desculpa, mas hoje a felicidade chegou primeiro!
 D1 - Feliz é aquele que vê a felicidade dos outros sem ter inveja. O sol é para todos e a sombra pra quem merece.
 D2 - Minha meta é ser feliz, não perfeito.
 D3 - Ser feliz até onde der, até onde puder. Sem adiar, ser feliz o tanto que durar.
 D4 - Nunca deixe ninguém dizer que você não pode fazer alguma coisa. Se você tem um sonho, tem que correr atrás dele. As pessoas não conseguem vencer, e dizem que você também não vai vencer. Se quer alguma coisa, corre atrás.
 D5 - O maior problema em acreditar nas pessoas erradas, é que um dia você acaba não acreditando em mais ninguém.
 D6 - Não me deixe ir, posso nunca mais voltar.
 D7 - Não me 

,POSITIVO,NEGATIVO,NEUTRO
D0,-20.82,-20.28,-39.38
D1,-1.82,-51.84,7.89
D2,-29.84,-36.44,-47.39
D3,-33.92,-66.21,-55.88
D4,22.07,-6.30,33.16
D5,-41.34,-31.29,-51.93
D6,-50.94,-38.56,-50.73
D7,28.52,-29.86,29.32
D8,-14.51,-14.35,-17.27
D9,-53.66,-38.33,-42.17


In [8]:
print('Loading emotional words: ')
emotion_words = load_valence_emotions_from_sentilex(
        '/home/rdenadai/programas/vagrant_img/vagrant.machine.puphpet/projetos/github/sentiment-analysis-2018-president-election/dataset/emocoes/SentiLex-PT02/SentiLex-flex-PT02.txt')

print('Starting EmotionalLSA model...')
model = EmotionalLSA(debug=True)
print('fit and transform...')
predicted = model.fit_transform(phrases, emotion_words)

print('\nShowing results:')
print('-' * 40)
for i, frase in enumerate(original_phrases):
    print(f' D{i} - {frase}')

size = len(original_phrases)
df = pd.DataFrame(predicted[:size], index=ldocs, columns=emotion_words.keys())
display(df)

Loading emotional words: 
Starting EmotionalLSA model...
fit and transform...
Count:
Actual number of features: 6313
Processing emotional state... this may take a while...

Showing results:
----------------------------------------
 D0 - Quando a tristeza bater na sua porta, abra um belo sorriso e diga: Desculpa, mas hoje a felicidade chegou primeiro!
 D1 - Feliz é aquele que vê a felicidade dos outros sem ter inveja. O sol é para todos e a sombra pra quem merece.
 D2 - Minha meta é ser feliz, não perfeito.
 D3 - Ser feliz até onde der, até onde puder. Sem adiar, ser feliz o tanto que durar.
 D4 - Nunca deixe ninguém dizer que você não pode fazer alguma coisa. Se você tem um sonho, tem que correr atrás dele. As pessoas não conseguem vencer, e dizem que você também não vai vencer. Se quer alguma coisa, corre atrás.
 D5 - O maior problema em acreditar nas pessoas erradas, é que um dia você acaba não acreditando em mais ninguém.
 D6 - Não me deixe ir, posso nunca mais voltar.
 D7 - Não me 

,POSITIVO,NEGATIVO,NEUTRO
D0,-19.97,-37.41,-50.78
D1,-35.92,-8.00,-51.26
D2,-36.43,-56.80,-36.96
D3,-65.84,-57.55,-49.95
D4,2.83,7.28,-41.31
D5,-40.38,-48.80,-35.91
D6,-64.17,-57.75,-47.14
D7,-36.53,19.99,-49.30
D8,-33.34,-14.73,-64.56
D9,-42.70,-50.89,-62.86
